In [1]:
!gdown 1XGLAnvZt9oEYCscQcqHjbva0k5S605BZ
!gdown 1EEBfIUZD0q0wRXo8pPFnIGzlzaexI6si

Downloading...
From: https://drive.google.com/uc?id=1XGLAnvZt9oEYCscQcqHjbva0k5S605BZ
To: /content/test_data.csv
100% 176k/176k [00:00<00:00, 70.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EEBfIUZD0q0wRXo8pPFnIGzlzaexI6si
To: /content/train_data.csv
100% 692k/692k [00:00<00:00, 38.7MB/s]


In [2]:
import numpy as np
import pandas as pd
import gensim
import gensim.downloader as api
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [3]:
train_df = pd.read_csv("train_data.csv")
X_train = train_df['clean_text'].to_numpy()
y_train = train_df['target'].to_numpy()

In [4]:
test_df = pd.read_csv("test_data.csv")
X_test = test_df['clean_text'].to_numpy()
y_test = test_df['target'].to_numpy()

# Pretrained Word2Vec

In [ ]:
path = api.load("fasttext-wiki-news-subwords-300", return_path=True)
print(path)

[==================================================] 100.0% 958.5/958.4MB downloaded
/root/gensim-data/fasttext-wiki-news-subwords-300/fasttext-wiki-news-subwords-300.gz


In [ ]:
model_fasttext = gensim.models.KeyedVectors.load_word2vec_format(path)

In [ ]:
voc = set(model_fasttext.index_to_key)
def document_vector(doc):
    doc = doc.split()
    doc = [word for word in doc if word in voc]
    return np.mean(model_fasttext[doc], axis=0)

In [ ]:
X_train_fasttext = []
for review in X_train:
    X_train_fasttext.append(document_vector(review))

In [ ]:
X_test_fasttext = []
for review in X_test:
    X_test_fasttext.append(document_vector(review))

In [ ]:
np.array(X_train_fasttext).shape

(2842, 300)

# Training and Testing

In [ ]:
def training_and_prediction(training_model, X_train, X_test, y_train, y_test):
  model = training_model
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  # print("Accuracy =", accuracy)
  # print("Confusion Matrix =\n", confusion_matrix(y_test, y_pred))
  # print(classification_report(y_test, y_pred))
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

In [ ]:
def get_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

## Naive Bayes

In [ ]:
scaler = MinMaxScaler()
training_and_prediction(MultinomialNB(), scaler.fit_transform(X_train_fasttext), scaler.fit_transform(X_test_fasttext), y_train, y_test)

{'Accuracy': 0.6905766526019691,
 'Precision': 0.6984126984126984,
 'Recall': 0.6376811594202898,
 'F1-Score': 0.6666666666666666}

In [ ]:
training_and_prediction(GaussianNB(), X_train_fasttext, X_test_fasttext, y_train, y_test)

{'Accuracy': 0.6905766526019691,
 'Precision': 0.654320987654321,
 'Recall': 0.7681159420289855,
 'F1-Score': 0.7066666666666667}

In [ ]:
training_and_prediction(BernoulliNB(), X_train_fasttext, X_test_fasttext, y_train, y_test)

{'Accuracy': 0.6849507735583685,
 'Precision': 0.6613333333333333,
 'Recall': 0.7188405797101449,
 'F1-Score': 0.6888888888888889}

## SVM

In [ ]:
training_and_prediction(SVC(C=16), X_train_fasttext, X_test_fasttext, y_train, y_test)

{'Accuracy': 0.749648382559775,
 'Precision': 0.7275204359673024,
 'Recall': 0.7739130434782608,
 'F1-Score': 0.7499999999999999}

In [ ]:
training_and_prediction(SVC(kernel='poly', C=6.5), X_train_fasttext, X_test_fasttext, y_train, y_test)

{'Accuracy': 0.7510548523206751,
 'Precision': 0.7386363636363636,
 'Recall': 0.7536231884057971,
 'F1-Score': 0.7460545193687231}

## Random Forest

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train_fasttext, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best Parameters: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 300}
Best Score: 0.7227305266315867


In [ ]:
training_and_prediction(RandomForestClassifier(n_estimators=300, max_depth=10), X_train_fasttext, X_test_fasttext, y_train, y_test)

{'Accuracy': 0.710267229254571,
 'Precision': 0.671604938271605,
 'Recall': 0.7884057971014493,
 'F1-Score': 0.7253333333333333}

## Logistic Regression

In [ ]:
# Define the parameter grid
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

# Create a Logistic Regression classifier
logistic_reg = LogisticRegression()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=logistic_reg, param_grid=param_grid, cv=3, scoring='f1', verbose=2, n_jobs=-1)

# Perform grid search
grid_search.fit(X_train_fasttext, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best Parameters: {'C': 10, 'penalty': 'l2', 'solver': 'saga'}
Best Score: 0.7653482608610608


In [ ]:
training_and_prediction(LogisticRegression(penalty='l2', C=10, solver='saga'), X_train_fasttext, X_test_fasttext, y_train, y_test)

{'Accuracy': 0.7341772151898734,
 'Precision': 0.7142857142857143,
 'Recall': 0.7536231884057971,
 'F1-Score': 0.7334273624823695}

## Gradient Boosting

In [ ]:
# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'subsample': [0.5, 0.7, 1.0],
}

# Create an XGBoost classifier
xgb_classifier = XGBClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=2, scoring='f1', verbose=2, n_jobs=-1)

# Perform grid search
grid_search.fit(X_train_fasttext, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 2 folds for each of 81 candidates, totalling 162 fits
Best Parameters: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.7}
Best Score: 0.7559625843473368


In [ ]:
training_and_prediction(XGBClassifier(learning_rate=0.01, max_depth=5, n_estimators=300, subsample=0.7), X_train_fasttext, X_test_fasttext, y_train, y_test)

{'Accuracy': 0.7130801687763713,
 'Precision': 0.6784810126582278,
 'Recall': 0.7768115942028986,
 'F1-Score': 0.7243243243243245}

## Deep Learning Techniques with Word2vec

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Bidirectional, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Tokenize text data
tokenizer = Tokenizer(num_words=7000)
tokenizer.fit_on_texts(np.concatenate((X_train, X_test), axis=0))
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
high = 0
for temp in X_train_sequences:
  high = max(high, len(temp))
high

125

In [ ]:
# Pad sequences to same length
max_len = 150
X_train_padded_sequences = pad_sequences(X_train_sequences, maxlen=max_len)
X_test_padded_sequences = pad_sequences(X_test_sequences, maxlen=max_len)

In [ ]:
# Create embedding layer
embedding_dim = 300
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in model_fasttext:
        embedding_matrix[i] = model_fasttext[word]
embedding_layer = Embedding(len(tokenizer.word_index) + 1, embedding_dim, weights=[embedding_matrix], trainable=False)

### ANN

In [ ]:
def ann_models(num_neurons_layer1, num_neurons_layer2=0, dropout_rate=0):
    model_dense = Sequential()
    model_dense.add(embedding_layer)
    model_dense.add(GlobalAveragePooling1D())
    model_dense.add(Dense(num_neurons_layer1, activation='relu'))
    model_dense.add(Dropout(dropout_rate))
    if num_neurons_layer2 > 0:
      model_dense.add(Dense(num_neurons_layer2, activation='relu'))
    model_dense.add(Dense(1, activation='sigmoid'))
    model_dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_dense.fit(X_train_padded_sequences, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_padded_sequences, y_test))
    y_pred_dense = model_dense.predict(X_test_padded_sequences)
    y_pred_dense = (y_pred_dense > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'num_neurons_layer2': num_neurons_layer2, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_dense)]


In [ ]:
for num_neurons_layer1 in [128, 64]:
  for num_neurons_layer2 in [64, 32, 0]:
    for dropout_rate in [0.0, 0.1, 0.2, 0.3, 0.4]:
      print(ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate))

23/23 [==============================] - 0s 8ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.0}, {'Accuracy': 0.7440225035161744, 'Precision': 0.7492354740061162, 'Recall': 0.7101449275362319, 'F1-Score': 0.7291666666666667}]
23/23 [==============================] - 0s 9ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.1}, {'Accuracy': 0.749648382559775, 'Precision': 0.7420289855072464, 'Recall': 0.7420289855072464, 'F1-Score': 0.7420289855072464}]
23/23 [==============================] - 0s 5ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.2}, {'Accuracy': 0.7566807313642757, 'Precision': 0.7514619883040936, 'Recall': 0.744927536231884, 'F1-Score': 0.7481804949053859}]
23/23 [==============================] - 0s 6ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.3}, {'Accuracy': 0.7538677918424754, 'Precision': 0.7401129943502824, 'Recall': 0.7594202898550725,

### LSTM

In [ ]:
def lstm_models(num_neurons_layer1, dropout_rate=0):
    model_lstm = Sequential()
    model_lstm.add(embedding_layer)
    model_lstm.add(LSTM(128, dropout=dropout_rate))
    model_lstm.add(Dense(1, activation='sigmoid'))
    model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model_lstm.fit(X_train_padded_sequences, y_train, epochs=20, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_padded_sequences, y_test))
    y_pred_lstm = model_lstm.predict(X_test_padded_sequences)
    y_pred_lstm = (y_pred_lstm > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_lstm)]

In [ ]:
for num_neurons_layer1 in [128, 64]:
  for dropout_rate in [0.0, 0.1, 0.2, 0.3, 0.4]:
    print(lstm_models(num_neurons_layer1, dropout_rate))

23/23 [==============================] - 3s 88ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.0}, {'Accuracy': 0.7468354430379747, 'Precision': 0.7391304347826086, 'Recall': 0.7391304347826086, 'F1-Score': 0.7391304347826085}]
23/23 [==============================] - 2s 84ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.1}, {'Accuracy': 0.739803094233474, 'Precision': 0.7040816326530612, 'Recall': 0.8, 'F1-Score': 0.7489823609226594}]
23/23 [==============================] - 2s 86ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.2}, {'Accuracy': 0.7355836849507735, 'Precision': 0.7198879551820728, 'Recall': 0.744927536231884, 'F1-Score': 0.7321937321937321}]
23/23 [==============================] - 2s 88ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.3}, {'Accuracy': 0.739803094233474, 'Precision': 0.7150537634408602, 'Recall': 0.7710144927536232, 'F1-Score': 0.7419804741980476}]
23/23 [==============================] - 2s 89ms/step
[{'num_neurons_layer1': 128,

### Bidirectional LSTM

In [ ]:
def bilstm_models(num_neurons_layer1, spatial_dropout_rate=0, dropout_rate=0):
    model_bilstm = Sequential()
    model_bilstm.add(embedding_layer)
    model_bilstm.add(SpatialDropout1D(spatial_dropout_rate))
    model_bilstm.add(Bidirectional(LSTM(num_neurons_layer1, return_sequences=False)))
    model_bilstm.add(Dropout(dropout_rate))
    model_bilstm.add(Dense(1, activation='sigmoid'))
    model_bilstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model_bilstm.fit(X_train_padded_sequences, y_train, epochs=10, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_padded_sequences, y_test))
    y_pred_bilstm = model_bilstm.predict(X_test_padded_sequences)
    y_pred_bilstm = (y_pred_bilstm > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'spatial_dropout_rate': spatial_dropout_rate, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_bilstm)]

In [ ]:
for num_neurons_layer1 in [128, 64]:
  for spatial_dropout_rate in [0, 0.1, 0.2, 0.3, 0.4]:
    for dropout_rate in [0, 0.1, 0.2, 0.3, 0.4]:
      print(bilstm_models(num_neurons_layer1, spatial_dropout_rate, dropout_rate))

23/23 [==============================] - 6s 207ms/step
[{'num_neurons_layer1': 128, 'spatial_dropout_rate': 0, 'dropout_rate': 0}, {'Accuracy': 0.7369901547116737, 'Precision': 0.7438271604938271, 'Recall': 0.6985507246376812, 'F1-Score': 0.7204783258594918}]
23/23 [==============================] - 6s 218ms/step
[{'num_neurons_layer1': 128, 'spatial_dropout_rate': 0, 'dropout_rate': 0.1}, {'Accuracy': 0.7369901547116737, 'Precision': 0.7515923566878981, 'Recall': 0.6840579710144927, 'F1-Score': 0.7162367223065251}]
23/23 [==============================] - 6s 179ms/step
[{'num_neurons_layer1': 128, 'spatial_dropout_rate': 0, 'dropout_rate': 0.2}, {'Accuracy': 0.7327707454289732, 'Precision': 0.7023498694516971, 'Recall': 0.7797101449275362, 'F1-Score': 0.7390109890109889}]
23/23 [==============================] - 5s 183ms/step
[{'num_neurons_layer1': 128, 'spatial_dropout_rate': 0, 'dropout_rate': 0.3}, {'Accuracy': 0.7426160337552743, 'Precision': 0.7237569060773481, 'Recall': 0.75942

### GRU

In [ ]:
model_gru = Sequential()
model_gru.add(embedding_layer)
# model_gru.add(SpatialDropout1D(0.2))
model_gru.add(GRU(64, return_sequences = False))
model_gru.add(Dropout(0.2))
model_gru.add(Dense(1, activation = 'sigmoid'))
model_gru.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         3080400   
                                                                 
 gru_8 (GRU)                 (None, 64)                70272     
                                                                 
 dropout_8 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3150737 (12.02 MB)
Trainable params: 70337 (274.75 KB)
Non-trainable params: 3080400 (11.75 MB)
_________________________________________________________________


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_gru.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_gru.fit(X_train_padded_sequences, y_train, epochs=50, batch_size=32, validation_data=(X_test_padded_sequences, y_test), callbacks=[early_stop])

Epoch 1/50
89/89 [==============================] - 14s 138ms/step - loss: 0.6688 - accuracy: 0.5767 - val_loss: 0.5921 - val_accuracy: 0.6709
Epoch 2/50
89/89 [==============================] - 11s 126ms/step - loss: 0.5649 - accuracy: 0.7108 - val_loss: 0.5442 - val_accuracy: 0.7145
Epoch 3/50
89/89 [==============================] - 12s 137ms/step - loss: 0.5177 - accuracy: 0.7498 - val_loss: 0.5245 - val_accuracy: 0.7314
Epoch 4/50
89/89 [==============================] - 11s 127ms/step - loss: 0.5076 - accuracy: 0.7572 - val_loss: 0.5205 - val_accuracy: 0.7300
Epoch 5/50
89/89 [==============================] - 11s 125ms/step - loss: 0.4918 - accuracy: 0.7653 - val_loss: 0.5365 - val_accuracy: 0.7412
Epoch 6/50
89/89 [==============================] - 12s 132ms/step - loss: 0.4745 - accuracy: 0.7762 - val_loss: 0.5149 - val_accuracy: 0.7426
Epoch 7/50
89/89 [==============================] - 10s 116ms/step - loss: 0.4830 - accuracy: 0.7695 - val_loss: 0.5254 - val_accuracy: 0.7342

In [ ]:
y_pred_gru = model_gru.predict(X_test_padded_sequences)
y_pred_gru = (y_pred_gru > 0.5).astype(int)
get_report(y_test, y_pred_gru)

23/23 [==============================] - 3s 88ms/step


{'Accuracy': 0.7426160337552743,
 'Precision': 0.7314285714285714,
 'Recall': 0.7420289855072464,
 'F1-Score': 0.7366906474820144}